In [29]:
import sys
sys.path.append('/cs/student/klee/ve/lib/python2.7/site-packages/')

import sys
import random
from sklearn.metrics import precision_recall_curve
import cPickle
# import _pickle as cPickle

from gensim.models import word2vec
from gensim import corpora, models, similarities
import logging
import pdb
import numpy as np
from tqdm import tqdm
import re
from collections import Counter
from matplotlib import pyplot as plt
from sklearn.metrics import average_precision_score
import numpy as np

tqdm.monitor_interval = 0

dataset = 'database'
workspaceDir = '/scratch/home/hwzha/workspace'
query = 'xml_databases'.replace(' ', '_')
TOPK = 50
information_retrievalDir = '%s/evaluation/%s/information_retrieval' % (workspaceDir, dataset)

APPROACHES = ['auto', 'textrank', 'kea', 'rake', 'spacy_np', 'StructMineDataPipeline', 'econ']

# visualize segmentations of each approach
approach2segmentation = {}
for approach in APPROACHES:
    textFile = '/scratch/home/hwzha/workspace/%s/result/%s/merged.txt_without_sentence_id.evaluation.txt' % (approach, dataset)
    with open(textFile) as fin:
        approach2segmentation[approach] = [i for i in fin]

lineCount = len(approach2segmentation[approach])

for visualize_count in np.random.randint(lineCount, size=5):
    print(visualize_count)
    for approach in ['auto', 'econ']:
        if approach == 'auto':
            if '<c>' not in approach2segmentation[approach][visualize_count]:
                break
        print(approach)
        print(approach2segmentation[approach][visualize_count])
    print('--'*20)


def retrieve(query, approach, approach2segmentation=approach2segmentation, topN=None):
    if approach in ['StructMineDataPipeline', 'rake', 'econ', 'kea']:
        query_re = re.compile(r'<c>%s<\/c>' % query)
    else:
        query_re = re.compile('\b%s\b' % query)
    # wrapped_query_re = re.crompile(r'<c>%s<\/c>' % query)
    if topN:
        return [i for i, l in enumerate(approach2segmentation[approach]) if query_re.search(l)][:topN]

    return [i for i, l in enumerate(approach2segmentation[approach]) if query_re.search(l)]

# information retrieval

# texts
textFile = '/scratch/home/hwzha/data/%s/merged.txt_without_sentence_id' % dataset
with open(textFile) as fin:
    texts = [i for i in fin]

# now retrieve score list of all documents
def retrieve_scores(query, file_pureText, dictionary, modelTfidf):
    index = cPickle.load(open(file_pureText+'.index', 'rb'))
    scores_matrix = index[modelTfidf[dictionary.doc2bow([query])]]
    scores_matrix = np.array(scores_matrix)
    return scores_matrix

def remove_marker(text):
    return re.subn(pattern='</?c>',string=text, repl='')[0]
def wrap_marker(text):
    text = re.subn(pattern='</?c>',string=text, repl='')[0]
    return '<c>' + text + '</c>'

def get_line_count(inFile):
    count = -1
    for count, line in enumerate(open(inFile, 'r')):
        pass
    count += 1
    return count

82011
----------------------------------------
293632
auto
The hierarchical <c>pattern_graph</c> , HP G Q V , E , AN , is a <c>labeled_directed_graph</c> with V the nodes , E the <c>direct_edges</c> and AN the annotations associated with edges .

econ
The <c>graph</c> <c>Q_V</c> <c>E</c> <c>AN</c> is a labeled directed <c>graph</c> with <c>V</c> the <c>nodes</c> <c>E</c> the <c>direct_edges</c> and AN the <c>annotations</c> associated with <c>edges</c>

----------------------------------------
1240849
----------------------------------------
1588732
----------------------------------------
1312795
----------------------------------------


In [27]:
APPROACHES = ['auto', 'textrank', 'kea', 'rake', 'spacy_np', 'StructMineDataPipeline', 'econ']
approach2indexes = {}

for approach in tqdm(APPROACHES):
    # if approach in ['StructMineDataPipeline', 'rake', 'econ', 'kea']:
    #     query = wrap_marker(query)

    file_pureText = "%s/%s/result/%s/merged.txt_without_sentence_id.evaluation.txt" % (workspaceDir, approach, dataset)
    corpus = corpora.MmCorpus(file_pureText + '.corpus')
    dictionary = corpora.Dictionary.load(file_pureText + '.dict')
    modelTfidf = models.TfidfModel.load(file_pureText + '.modelTfidf')

    # scores_matrix = retrieve(query, file_pureText=file_pureText, dictionary=dictionary, modelTfidf=modelTfidf)
    # retreived_indexes = scores_matrix.argsort()[-TOPK:][::-1] # first pass retrieved indexes
    retreived_indexes = retrieve(query, approach, topN=TOPK)
    approach2indexes[approach] = retreived_indexes
    # query = remove_marker(query)

common_indexes = set.intersection(*[set(approach2indexes[approach]) for approach in APPROACHES])
all_retreived_indexes = []
for approach in APPROACHES:
    retreived_indexes = [index for index in approach2indexes[approach] if index not in common_indexes]
    all_retreived_indexes.extend(retreived_indexes)
all_retreived_indexes = list(set(all_retreived_indexes))

#second pass retrieve scores
approach2scores = {}
for approach in tqdm(APPROACHES):
    if approach in ['StructMineDataPipeline', 'rake', 'econ', 'kea']:
        query = wrap_marker(query)

    file_pureText = "%s/%s/result/%s/merged.txt_without_sentence_id.evaluation.txt" % (workspaceDir, approach, dataset)
    corpus = corpora.MmCorpus(file_pureText + '.corpus')
    dictionary = corpora.Dictionary.load(file_pureText + '.dict')
    modelTfidf = models.TfidfModel.load(file_pureText + '.modelTfidf')
    scores_matrix = retrieve_scores(query, file_pureText, dictionary=dictionary, modelTfidf=modelTfidf)
    all_scores = scores_matrix[all_retreived_indexes] # scores of all_retreived_indexes
    approach2scores[approach] = all_scores
    query = remove_marker(query)

with open('%s/%s_%s.txt' % (information_retrievalDir, query, TOPK), 'w') as f_out, \
    open('%s/%s_%s_explanation.txt' % (information_retrievalDir, query, TOPK), 'w') as f_out_explanation:
        for i, index in enumerate(all_retreived_indexes):
            score_explanation = []
            for approach in APPROACHES:
                score_explanation.append(approach + ':' + str(approach2scores[approach][i]))
            score_explanation = ','.join(score_explanation)
            f_out.write(texts[index])
            f_out_explanation.write(texts[index].strip() + '\t' + str(index) + '\t' + score_explanation + '\n')

with open('%s/%s_%s_order.txt' % (information_retrievalDir, query, TOPK), 'w') as f_out_order:
    indexs = [str(i) for i in all_retreived_indexes]
    f_out_order.write(','.join(indexs) + '\n')
    for approach in APPROACHES:
        scores = approach2scores[approach].tolist()
        scores = [str(s) for s in scores]
        f_out_order.write(approach + '\t'  + ','.join(scores) + '\n')

100%|██████████| 7/7 [00:46<00:00,  6.66s/it]


In [35]:
raw_autophrase_output = "/scratch/home/hwzha/workspace/AutoPhrase/models/%s/segmentation.txt" % (dataset)
auto_text = []
with open(raw_autophrase_output) as f:
    for line in f:
        auto_text.append(line.replace('<phrase>', '<c>').replace('</phrase>', '</c>'))
approach2segmentation['auto'] = auto_text

In [50]:
def visualize_by_index(index):
    for approach in ['auto', 'econ', 'StructMineDataPipeline']:
        if approach == 'auto':
            if '<c>' not in approach2segmentation[approach][index]:
                break
        print(approach)
        print(approach2segmentation[approach][index])
    print('--'*20)

for visualize_count in np.random.randint(lineCount, size=5):
    print(visualize_count)
    visualize_by_index(visualize_count)

1608407
auto
We first introduce a new concept , called subgraph <c>core number</c> , which is geared to the main idea of Inc-S .

econ
We first introduce a <c>new_concept</c> called <c>core_number</c> which is geared to the <c>main_idea</c> of <c>Inc-S</c>

StructMineDataPipeline
We first introduce a <c>new_concept</c> , called <c>subgraph_core_number</c> , which is geared to the <c>main_idea</c> of <c>Inc-S</c> .

----------------------------------------
1434997
auto
5 3 <c>Solver Derivation</c> Rules

econ
<c>Solver_Derivation_Rules</c>

StructMineDataPipeline
5 3 <c>Solver_Derivation_Rules</c>

----------------------------------------
165085
----------------------------------------
1144595
----------------------------------------
1265520
auto
<c>Snapshot Isolation SI</c> has become a popular <c>isolation level</c> in database systems .

econ
<c>SI</c> has become a <c>isolation_level</c> in <c>database_systems</c>

StructMineDataPipeline
<c>Snapshot_Isolation_SI</c> has become a <c>p

In [51]:
[(i,l) for i,l in enumerate(approach2segmentation['auto']) if l.startswith('Graphs-at-a-time')]

[(363059,
  'Graphs-at-a-time : <c>Query Language</c> and Access Methods for <c>Graph Databases</c>\n')]

In [52]:
visualize_by_index(363059)

auto
Graphs-at-a-time : <c>Query Language</c> and Access Methods for <c>Graph Databases</c>

econ
<c>Graphs-at-a-time</c> <c>Query_Language</c> and <c>Access_Methods</c> for <c>Graph_Databases</c>

StructMineDataPipeline
<c>Graphs-at-a-time</c> : <c>Query_Language</c> and <c>Access_Methods</c> for <c>Graph_Databases</c>

----------------------------------------
